importing packages

In [1]:
import numpy as np
import string 
import scipy.io
import os
import re

import nilearn.signal

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Reading brain Data:

In [2]:
subject_id = "1"
datadir = "../data/"

fMRI_preprocessing_mode = "nothing"#"mean" 
fMRI_preprocessing_flags= {"none":{'detrend':False, 'standard':False,'highpass':None},
                     "detrend":{'detrend':True, 'standard':False,'highpass':None},
                     "detrend_filter":{'detrend':True, 'standard':False,'highpass':0.005},
                     "detrend_filter_std":{'detrend':True, 'standard':True,'highpass':0.005}}

subject_file = scipy.io.loadmat(datadir+'subject_'+subject_id+'.mat') 

sub_id_index = 0
#nTRs
number_of_scans_index = 1
#nvoxels
number_of_voxels = 2
x_dim_index = 3
y_dim_index = 4
z_dim_index = 5
#colToCoord
colToCoord_index = 6
#coordToCol
coordToCol_index = 7
ROInumToName_index = 8
ROInumsToName_3d_index = 9
ROINames_index = 10
voxel_size_index = 11
#the map to the MNI space
matrix_index = 12

activations = subject_file['data']
subject_id = subject_file['meta'][0][0][sub_id_index][0][0]
number_of_scans = subject_file['meta'][0][0][number_of_scans_index][0][0]
number_of_voxels = subject_file['meta'][0][0][number_of_voxels][0][0]
x_dim = subject_file['meta'][0][0][x_dim_index][0][0]
y_dim = subject_file['meta'][0][0][y_dim_index][0][0]
z_dim = subject_file['meta'][0][0][z_dim_index][0][0]
coords = subject_file['meta'][0][0][colToCoord_index]
named_areas = subject_file['meta'][0][0][ROInumToName_index]
column_map = subject_file['meta'][0][0][coordToCol_index].shape
area_names_list = subject_file['meta'][0][0][ROINames_index][0]
voxel_size = subject_file['meta'][0][0][voxel_size_index]
nmi_matrix = subject_file['meta'][0][0][matrix_index]

Reading Story Feature:

In [3]:
index = 0
word_index = 0
time_index = 1
subject_file['words'][0][index][word_index][0][0][0]
story_features = scipy.io.loadmat('../data/story_features.mat') 
part_of_speaches_feature_id = 8
speach_feature_id = 1
part_of_speaches = story_features['features'][0][part_of_speaches_feature_id][1][0]
part_of_speaches_features = story_features['features'][0][part_of_speaches_feature_id][2]
speaches = story_features['features'][0][speach_feature_id][1][0]
speach_features = story_features['features'][0][speach_feature_id][2]

In [4]:
speaches

array([array(['speak_sticky'], dtype='<U12'),
       array(['speak'], dtype='<U5')], dtype=object)

In [5]:
print("How many words are there in the story? aligned with the brain data?")
subject_file['words'].shape

How many words are there in the story? aligned with the brain data?


(1, 5176)

In [6]:
actual_words = []
word_times = []
word_pos = []
speach_states = []
for i in np.arange(subject_file['words'].shape[1]):
    actual_words.append(subject_file['words'][0][i][word_index][0][0][0])
    word_times.append(subject_file['words'][0][i][time_index][0][0])
    word_pos.append(part_of_speaches_features[i])
    speach_states.append(speach_features[i])

Group time steps into blocks according to the fMRI experiment:

In [7]:
blocks = subject_file['time'][:,1]
scan_times = subject_file['time'][:,0]

block_ends = {}
block_ends_indexes = {}


for block_id in np.arange(1,5):
    block_ends_indexes[block_id] = np.where(scan_times == np.max(scan_times[np.where(blocks == block_id )]))[0]
    block_ends[block_id] = scan_times[block_ends_indexes[block_id]]+2


print(block_ends)
print(block_ends_indexes)

{1: array([680], dtype=uint16), 2: array([1384], dtype=uint16), 3: array([1942], dtype=uint16), 4: array([2702], dtype=uint16)}
{1: array([339]), 2: array([691]), 3: array([970]), 4: array([1350])}


group words and story features into blocks:

In [8]:
block_texts = {1:[],2:[],3:[],4:[]}
block_pos = {1:[],2:[],3:[],4:[]}
block_speach_state = {1:[],2:[],3:[],4:[]}
block_steps = {1:[],2:[],3:[],4:[]}
block_id = 1
for index in np.arange(len(actual_words)):
    if word_times[index] > block_ends[block_id]:
        block_id += 1
    block_texts[block_id].append(str(actual_words[index].encode("ascii",'ignore').decode()))
    block_pos[block_id].append(word_pos[index])
    block_speach_state[block_id].append(speach_states[index])
    block_steps[block_id].append(index)

In [9]:
len(list(zip(block_steps[1],block_texts[1])))

1303

Prepare input stimuli for the computational model (character input language model):

In [10]:
block_sentences = {1:[],2:[],3:[],4:[]}
translated_block_texts = {1:[],2:[],3:[],4:[]}
translated_block_pos = {1:[],2:[],3:[],4:[]}
translated_block_texts_steps = {1:[],2:[],3:[],4:[]}
translator = str.maketrans('', '', string.punctuation)


for block_id in np.arange(1,5):
    new_sentence = []
    for word,step,pos in zip(block_texts[block_id],block_steps[block_id],block_pos[block_id]):
        splitted_word = re.split("([A-Za-z\']+)", word.strip())
        tranlated_word = word.translate(translator)
        print("step:",step, "splited word:",splitted_word)
        last_w = ""
        for w in splitted_word:
            if len(w) > 0:
                last_w = w
                new_sentence.append(w)
                translated_block_texts[block_id].append(w)
                translated_block_texts_steps[block_id].append(step)
                translated_block_pos[block_id].append(pos)
        if last_w in ".?!":
            block_sentences[block_id].append(new_sentence)
            new_sentence = []
        
        """
        if len(word) == 1:
            tranlated_word = word
        if len(tranlated_word.strip()) > 0:
            translated_block_texts[block_id].append(tranlated_word)
            translated_block_texts_steps[block_id].append(step)
            translated_block_pos[block_id].append(pos)
            
        if len(tranlated_word.strip()) > 0:
            new_sentence.append(tranlated_word)
        if word.endswith("."):
            new_sentence.append(".")
            translated_block_texts[block_id].append(".")
            translated_block_texts_steps[block_id].append(step)
            translated_block_pos[block_id].append(np.zeros_like(pos))

            block_sentences[block_id].append(new_sentence)
            new_sentence = []
        """
    if len(new_sentence ) > 0:
            block_sentences[block_id].append(new_sentence)

    
    

step: 0 splited word: ['', 'Harry', '']
step: 1 splited word: ['', 'had', '']
step: 2 splited word: ['', 'never', '']
step: 3 splited word: ['', 'believed', '']
step: 4 splited word: ['', 'he', '']
step: 5 splited word: ['', 'would', '']
step: 6 splited word: ['', 'meet', '']
step: 7 splited word: ['', 'a', '']
step: 8 splited word: ['', 'boy', '']
step: 9 splited word: ['', 'he', '']
step: 10 splited word: ['', 'hated', '']
step: 11 splited word: ['', 'more', '']
step: 12 splited word: ['', 'than', '']
step: 13 splited word: ['', 'Dudley', ',']
step: 14 splited word: ['', 'but', '']
step: 15 splited word: ['', 'that', '']
step: 16 splited word: ['', 'was', '']
step: 17 splited word: ['', 'before', '']
step: 18 splited word: ['', 'he', '']
step: 19 splited word: ['', 'met', '']
step: 20 splited word: ['', 'Draco', '']
step: 21 splited word: ['', 'Malfoy', '.']
step: 22 splited word: ['', 'Still', ',']
step: 23 splited word: ['', 'first', '-', 'year', '']
step: 24 splited word: ['', 'Gr

step: 236 splited word: ['', 'families', '']
step: 237 splited word: ['', 'talked', '']
step: 238 splited word: ['', 'about', '']
step: 239 splited word: ['', 'Quidditch', '']
step: 240 splited word: ['', 'constantly', '.']
step: 241 splited word: ['', 'Ron', '']
step: 242 splited word: ['', 'had', '']
step: 243 splited word: ['', 'already', '']
step: 244 splited word: ['', 'had', '']
step: 245 splited word: ['', 'a', '']
step: 246 splited word: ['', 'big', '']
step: 247 splited word: ['', 'argument', '']
step: 248 splited word: ['', 'with', '']
step: 249 splited word: ['', 'Dean', '']
step: 250 splited word: ['', 'Thomas', ',']
step: 251 splited word: ['', 'who', '']
step: 252 splited word: ['', 'shared', '']
step: 253 splited word: ['', 'their', '']
step: 254 splited word: ['', 'dormitory', ',']
step: 255 splited word: ['', 'about', '']
step: 256 splited word: ['', 'soccer', '.']
step: 257 splited word: ['', 'Ron', '']
step: 258 splited word: ['', "couldn't", '']
step: 259 splited wo

step: 477 splited word: ['', 'a', '']
step: 478 splited word: ['', 'small', '']
step: 479 splited word: ['', 'package', '']
step: 480 splited word: ['', 'from', '']
step: 481 splited word: ['', 'his', '']
step: 482 splited word: ['', 'grandmother', '.']
step: 483 splited word: ['', 'He', '']
step: 484 splited word: ['', 'opened', '']
step: 485 splited word: ['', 'it', '']
step: 486 splited word: ['', 'excitedly', '']
step: 487 splited word: ['', 'and', '']
step: 488 splited word: ['', 'showed', '']
step: 489 splited word: ['', 'them', '']
step: 490 splited word: ['', 'a', '']
step: 491 splited word: ['', 'glass', '']
step: 492 splited word: ['', 'ball', '']
step: 493 splited word: ['', 'the', '']
step: 494 splited word: ['', 'size', '']
step: 495 splited word: ['', 'of', '']
step: 496 splited word: ['', 'a', '']
step: 497 splited word: ['', 'large', '']
step: 498 splited word: ['', 'marble', ',']
step: 499 splited word: ['', 'which', '']
step: 500 splited word: ['', 'seemed', '']
step:

step: 731 splited word: ['', 'there', ',']
step: 732 splited word: ['', 'and', '']
step: 733 splited word: ['', 'so', '']
step: 734 splited word: ['', 'were', '']
step: 735 splited word: ['', 'twenty', '']
step: 736 splited word: ['', 'broomsticks', '']
step: 737 splited word: ['', 'lying', '']
step: 738 splited word: ['', 'in', '']
step: 739 splited word: ['', 'neat', '']
step: 740 splited word: ['', 'lines', '']
step: 741 splited word: ['', 'on', '']
step: 742 splited word: ['', 'the', '']
step: 743 splited word: ['', 'ground', '.']
step: 744 splited word: ['', 'Harry', '']
step: 745 splited word: ['', 'had', '']
step: 746 splited word: ['', 'heard', '']
step: 747 splited word: ['', 'Fred', '']
step: 748 splited word: ['', 'and', '']
step: 749 splited word: ['', 'George', '']
step: 750 splited word: ['', 'Weasley', '']
step: 751 splited word: ['', 'complain', '']
step: 752 splited word: ['', 'about', '']
step: 753 splited word: ['', 'the', '']
step: 754 splited word: ['', 'school', '

step: 980 splited word: ['"', 'Keep', '']
step: 981 splited word: ['', 'your', '']
step: 982 splited word: ['', 'brooms', '']
step: 983 splited word: ['', 'steady', ',']
step: 984 splited word: ['', 'rise', '']
step: 985 splited word: ['', 'a', '']
step: 986 splited word: ['', 'few', '']
step: 987 splited word: ['', 'feet', ',']
step: 988 splited word: ['', 'and', '']
step: 989 splited word: ['', 'then', '']
step: 990 splited word: ['', 'come', '']
step: 991 splited word: ['', 'straight', '']
step: 992 splited word: ['', 'back', '']
step: 993 splited word: ['', 'down', '']
step: 994 splited word: ['', 'by', '']
step: 995 splited word: ['', 'leaning', '']
step: 996 splited word: ['', 'forward', '']
step: 997 splited word: ['', 'slightly', '.']
step: 998 splited word: ['', 'On', '']
step: 999 splited word: ['', 'my', '']
step: 1000 splited word: ['', 'whistle', '']
step: 1001 splited word: ['--']
step: 1002 splited word: ['', 'three', '']
step: 1003 splited word: ['--']
step: 1004 splite

...

step: 3354 splited word: ['', 'I', '']
step: 3355 splited word: ['', "don't", '']
step: 3356 splited word: ['', 'want', '']
step: 3357 splited word: ['', 'Slytherin', '']
step: 3358 splited word: ['', 'to', '']
step: 3359 splited word: ['', 'win', '']
step: 3360 splited word: ['', 'the', '']
step: 3361 splited word: ['', 'House', '']
step: 3362 splited word: ['', 'Cup', ',']
step: 3363 splited word: ['', 'and', '']
step: 3364 splited word: ['', "you'll", '']
step: 3365 splited word: ['', 'lose', '']
step: 3366 splited word: ['', 'all', '']
step: 3367 splited word: ['', 'the', '']
step: 3368 splited word: ['', 'points', '']
step: 3369 splited word: ['', 'I', '']
step: 3370 splited word: ['', 'got', '']
step: 3371 splited word: ['', 'from', '']
step: 3372 splited word: ['', 'Professor', '']
step: 3373 splited word: ['', 'McGonagall', '']
step: 3374 splited word: ['', 'for', '']
step: 3375 splited word: ['', 'knowing', '']
step: 3376 splited word: ['', 'about', '']
step: 3377 splited word

step: 3854 splited word: ['', 'and', '']
step: 3855 splited word: ['', 'gold', '']
step: 3856 splited word: ['', 'in', '']
step: 3857 splited word: ['', 'the', '']
step: 3858 splited word: ['', 'darkness', '.']
step: 3859 splited word: ['', 'They', '']
step: 3860 splited word: ['', 'edged', '']
step: 3861 splited word: ['', 'along', '']
step: 3862 splited word: ['', 'the', '']
step: 3863 splited word: ['', 'walls', ',']
step: 3864 splited word: ['', 'keeping', '']
step: 3865 splited word: ['', 'their', '']
step: 3866 splited word: ['', 'eyes', '']
step: 3867 splited word: ['', 'on', '']
step: 3868 splited word: ['', 'the', '']
step: 3869 splited word: ['', 'doors', '']
step: 3870 splited word: ['', 'at', '']
step: 3871 splited word: ['', 'either', '']
step: 3872 splited word: ['', 'end', '']
step: 3873 splited word: ['', 'of', '']
step: 3874 splited word: ['', 'the', '']
step: 3875 splited word: ['', 'room', '.']
step: 3876 splited word: ['', 'Harry', '']
step: 3877 splited word: ['', 

step: 4353 splited word: ['', 'you', '']
step: 4354 splited word: ['', "don't", '']
step: 4355 splited word: ['', 'give', '']
step: 4356 splited word: ['', 'us', '']
step: 4357 splited word: ['', 'away', ',']
step: 4358 splited word: ['', 'Peeves', ',']
step: 4359 splited word: ['', 'please', '."']
step: 4360 splited word: ['+']
step: 4361 splited word: ['"', 'Should', '']
step: 4362 splited word: ['', 'tell', '']
step: 4363 splited word: ['', 'Filch', ',']
step: 4364 splited word: ['', 'I', '']
step: 4365 splited word: ['', 'should', ',"']
step: 4366 splited word: ['', 'said', '']
step: 4367 splited word: ['', 'Peeves', '']
step: 4368 splited word: ['', 'in', '']
step: 4369 splited word: ['', 'a', '']
step: 4370 splited word: ['', 'saintly', '']
step: 4371 splited word: ['', 'voice', ',']
step: 4372 splited word: ['', 'but', '']
step: 4373 splited word: ['', 'his', '']
step: 4374 splited word: ['', 'eyes', '']
step: 4375 splited word: ['', 'glittered', '']
step: 4376 splited word: [''

step: 4728 splited word: ['', 'eyes', ';']
step: 4729 splited word: ['', 'three', '']
step: 4730 splited word: ['', 'noses', ',']
step: 4731 splited word: ['', 'twitching', '']
step: 4732 splited word: ['', 'and', '']
step: 4733 splited word: ['', 'quivering', '']
step: 4734 splited word: ['', 'in', '']
step: 4735 splited word: ['', 'their', '']
step: 4736 splited word: ['', 'direction', ';']
step: 4737 splited word: ['', 'three', '']
step: 4738 splited word: ['', 'drooling', '']
step: 4739 splited word: ['', 'mouths', ',']
step: 4740 splited word: ['', 'saliva', '']
step: 4741 splited word: ['', 'hanging', '']
step: 4742 splited word: ['', 'in', '']
step: 4743 splited word: ['', 'slippery', '']
step: 4744 splited word: ['', 'ropes', '']
step: 4745 splited word: ['', 'from', '']
step: 4746 splited word: ['', 'yellowish', '']
step: 4747 splited word: ['', 'fangs', '.']
step: 4748 splited word: ['+']
step: 4749 splited word: ['', 'It', '']
step: 4750 splited word: ['', 'was', '']
step: 4

step: 4978 splited word: ['', 'any', '']
step: 4979 splited word: ['', 'dog', '']
step: 4980 splited word: ['', 'needs', '']
step: 4981 splited word: ['', 'exercise', ',']
step: 4982 splited word: ['', 'that', '']
step: 4983 splited word: ['', 'one', '']
step: 4984 splited word: ['', 'does', '."']
step: 4985 splited word: ['+']
step: 4986 splited word: ['', 'Hermione', '']
step: 4987 splited word: ['', 'had', '']
step: 4988 splited word: ['', 'got', '']
step: 4989 splited word: ['', 'both', '']
step: 4990 splited word: ['', 'her', '']
step: 4991 splited word: ['', 'breath', '']
step: 4992 splited word: ['', 'and', '']
step: 4993 splited word: ['', 'her', '']
step: 4994 splited word: ['', 'bad', '']
step: 4995 splited word: ['', 'temper', '']
step: 4996 splited word: ['', 'back', '']
step: 4997 splited word: ['', 'again', '.']
step: 4998 splited word: ['+']
step: 4999 splited word: ['"', 'You', '']
step: 5000 splited word: ['', "don't", '']
step: 5001 splited word: ['', 'use', '']
step:

In [11]:
block_sentences[1] 

[['Harry',
  'had',
  'never',
  'believed',
  'he',
  'would',
  'meet',
  'a',
  'boy',
  'he',
  'hated',
  'more',
  'than',
  'Dudley',
  ',',
  'but',
  'that',
  'was',
  'before',
  'he',
  'met',
  'Draco',
  'Malfoy',
  '.'],
 ['Still',
  ',',
  'first',
  '-',
  'year',
  'Gryffindors',
  'only',
  'had',
  'Potions',
  'with',
  'the',
  'Slytherins',
  ',',
  'so',
  'they',
  "didn't",
  'have',
  'to',
  'put',
  'up',
  'with',
  'Malfoy',
  'much',
  '.'],
 ['Or',
  'at',
  'least',
  ',',
  'they',
  "didn't",
  'until',
  'they',
  'spotted',
  'a',
  'notice',
  'pinned',
  'up',
  'in',
  'the',
  'Gryffindor',
  'common',
  'room',
  'that',
  'made',
  'them',
  'all',
  'groan',
  '.'],
 ['Flying',
  'lessons',
  'would',
  'be',
  'starting',
  'on',
  'Thursday',
  '--',
  'and',
  'Gryffindor',
  'and',
  'Slytherin',
  'would',
  'be',
  'learning',
  'together',
  '.'],
 ['+', '"', 'Typical', ',"', 'said', 'Harry', 'darkly', '.'],
 ['"', 'Just', 'what', 'I'

In [12]:
block_text = {}
for block_id in np.arange(1,5):
    block_text[block_id] = ' '.join(translated_block_texts[block_id])
    
print(block_text)

{1: 'Harry had never believed he would meet a boy he hated more than Dudley , but that was before he met Draco Malfoy . Still , first - year Gryffindors only had Potions with the Slytherins , so they didn\'t have to put up with Malfoy much . Or at least , they didn\'t until they spotted a notice pinned up in the Gryffindor common room that made them all groan . Flying lessons would be starting on Thursday -- and Gryffindor and Slytherin would be learning together . + " Typical ," said Harry darkly . " Just what I always wanted . To make a fool of myself on a broomstick in front of Malfoy ." + He had been looking forward to learning to fly more than anything else . " You don\'t know that you\'ll make a fool of yourself ," said Ron reasonably . " Anyway , I know Malfoy\'s always going on about how good he is at Quidditch , but I bet that\'s all talk ." + Malfoy certainly did talk about flying a lot . He complained loudly about first years never getting on the House Quidditch teams and to

In [13]:
processed_data_dir = "../processed_data/subject_"+str(subject_id)
try:
    os.stat(processed_data_dir)
except:
    os.makedirs(processed_data_dir) 
for block_id in np.arange(1,5):
    
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_sentences",block_sentences[block_id])
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_text",translated_block_texts[block_id])
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_pos",translated_block_pos[block_id])

    with open(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_rawtext.txt","w") as fout:
        fout.write(block_text[block_id])

In [14]:
print(np.asarray(translated_block_texts[1])[np.where(np.asarray(translated_block_texts_steps[1]) == 117)])
print(np.asarray(translated_block_texts_steps[1])[np.where(np.asarray(translated_block_texts_steps[1]) == 117)])

["don't"]
[117]


In [15]:
translated_block_texts

{1: ['Harry',
  'had',
  'never',
  'believed',
  'he',
  'would',
  'meet',
  'a',
  'boy',
  'he',
  'hated',
  'more',
  'than',
  'Dudley',
  ',',
  'but',
  'that',
  'was',
  'before',
  'he',
  'met',
  'Draco',
  'Malfoy',
  '.',
  'Still',
  ',',
  'first',
  '-',
  'year',
  'Gryffindors',
  'only',
  'had',
  'Potions',
  'with',
  'the',
  'Slytherins',
  ',',
  'so',
  'they',
  "didn't",
  'have',
  'to',
  'put',
  'up',
  'with',
  'Malfoy',
  'much',
  '.',
  'Or',
  'at',
  'least',
  ',',
  'they',
  "didn't",
  'until',
  'they',
  'spotted',
  'a',
  'notice',
  'pinned',
  'up',
  'in',
  'the',
  'Gryffindor',
  'common',
  'room',
  'that',
  'made',
  'them',
  'all',
  'groan',
  '.',
  'Flying',
  'lessons',
  'would',
  'be',
  'starting',
  'on',
  'Thursday',
  '--',
  'and',
  'Gryffindor',
  'and',
  'Slytherin',
  'would',
  'be',
  'learning',
  'together',
  '.',
  '+',
  '"',
  'Typical',
  ',"',
  'said',
  'Harry',
  'darkly',
  '.',
  '"',
  'Just

In [16]:
translated_block_texts_steps[1]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 21,
 22,
 22,
 23,
 23,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 40,
 41,
 42,
 43,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 78,
 79,
 80,
 80,
 80,
 81,
 82,
 83,
 83,
 84,
 84,
 85,
 86,
 87,
 88,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 115,
 116,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 125,
 126,
 127,
 128,
 128,
 129,
 129,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 157,
 158,
 159,
 160,
 161,
 16

In [17]:
class Scan(object):
    def __init__(self,activations,timestamp, step,prev_words=None,next_words=None,all_words=None,all_pos=None,all_speak_features=None,current_translated_words=None):
        self.activations = activations
        self.timestamp = timestamp
        self.prev_words = prev_words
        self.next_words = next_words
        self.step = step
        self.all_words = all_words
        self.current_translated_words = current_translated_words
        self.all_pos = all_pos
        self.all_speak_features = all_speak_features
        self.brain3d = None
        


In [18]:
vocab = []
translated_words = []
with open(processed_data_dir+"/subject_"+subject_id+"_harry_potter_vocab.txt","w") as fout:
    for word in actual_words:
        w = word.encode("ascii",'ignore').decode()
        if len(w) > 1:
            w = w #.translate(translator)
        if len(w) > 0:
            fout.write(w+'\n')
            print(w)
            vocab.append(w)

        translated_words.append(w)
        

Harry
had
never
believed
he
would
meet
a
boy
he
hated
more
than
Dudley,
but
that
was
before
he
met
Draco
Malfoy.
Still,
first-year
Gryffindors
only
had
Potions
with
the
Slytherins,
so
they
didn't
have
to
put
up
with
Malfoy
much.
Or
at
least,
they
didn't
until
they
spotted
a
notice
pinned
up
in
the
Gryffindor
common
room
that
made
them
all
groan.
Flying
lessons
would
be
starting
on
Thursday
--
and
Gryffindor
and
Slytherin
would
be
learning
together.
+
"Typical,"
said
Harry
darkly.
"Just
what
I
always
wanted.
To
make
a
fool
of
myself
on
a
broomstick
in
front
of
Malfoy."
+
He
had
been
looking
forward
to
learning
to
fly
more
than
anything
else.
"You
don't
know
that
you'll
make
a
fool
of
yourself,"
said
Ron
reasonably.
"Anyway,
I
know
Malfoy's
always
going
on
about
how
good
he
is
at
Quidditch,
but
I
bet
that's
all
talk."
+
Malfoy
certainly
did
talk
about
flying
a
lot.
He
complained
loudly
about
first
years
never
getting
on
the
House
Quidditch
teams
and
told
long,
boastful
stories
that
alway

forward
and
pointed
his
broom
handle
down
--
next
second
he
was
gathering
speed
in
a
steep
dive,
racing
the
ball
--
wind
whistled
in
his
ears,
mingled
with
the
screams
of
people
watching
--
he
stretched
out
his
hand
--
a
foot
from
the
ground
he
caught
it,
just
in
time
to
pull
his
broom
straight,
and
he
toppled
gently
onto
the
grass
with
the
Remembrall
clutched
safely
in
his
fist.
+
"HARRY
POTTER!"
+
His
heart
sank
faster
than
he'd
just
dived.
Professor
McGonagall
was
running
toward
them.
He
got
to
his
feet,
trembling.
+
@"Never
--
in
all
my
time
at
Hogwarts
"
Professor
McGonagall
was
almost
speechless
with
shock,
and
her
glasses
flashed
furiously,
"
how
@dare
you
--
might
have
broken
your
neck
"
+
"It
wasn't
his
fault,
Professor
"
+
"Be
quiet,
Miss
Patil
"
+
"But
Malfoy
"
"That's
enough,
Mr.
Weasley.
Potter,
follow
me,
now."
+
Harry
caught
sight
of
Malfoy,
Crabbe,
and
Goyle's
triumphant
faces
as
he
left,
walking
numbly
in
Professor
McGonagall's
wake
as
she
strode
toward
the
castle.
He


the
darkness
--
this
was
his
big
chance
to
beat
Malfoy
face-to-face.
He
couldn't
miss
it.
+
"Half-past
eleven,"
Ron
muttered
at
last,
"we'd
better
go."
+
They
pulled
on
their
bathrobes,
picked
up
their
wands,
and
crept
across
the
tower
room,
down
the
spiral
staircase,
and
into
the
Gryffindor
common
room.
A
few
embers
were
still
glowing
in
the
fireplace,
turning
all
the
armchairs
into
hunched
black
shadows.
They
had
almost
reached
the
portrait
hole
when
a
voice
spoke
from
the
chair
nearest
them,
"I
can't
believe
you're
going
to
do
this,
Harry."
+
A
lamp
flickered
on.
It
was
Hermione
Granger,
wearing
a
pink
bathrobe
and
a
frown.
+
@"You!"
said
Ron
furiously.
"Go
back
to
bed!"
+
"I
almost
told
your
brother,"
Hermione
snapped,
"Percy
--
he's
a
prefect,
he'd
put
a
stop
to
this."
+
Harry
couldn't
believe
anyone
could
be
so
interfering.
+
"Come
on,"
he
said
to
Ron.
He
pushed
open
the
portrait
of
the
Fat
Lady
and
climbed
through
the
hole.
+
Hermione
wasn't
going
to
give
up
that
easily.
She
fol

nightmare
--
this
was
too
much,
on
top
of
everything
that
had
happened
so
far.
+
They
weren't
in
a
room,
as
he
had
supposed.
They
were
in
a
corridor.
The
forbidden
corridor
on
the
third
floor.
And
now
they
knew
why
it
was
forbidden.
+
They
were
looking
straight
into
the
eyes
of
a
monstrous
dog,
a
dog
that
filled
the
whole
space
between
ceiling
and
floor.
It
had
three
heads.
Three
pairs
of
rolling,
mad
eyes;
three
noses,
twitching
and
quivering
in
their
direction;
three
drooling
mouths,
saliva
hanging
in
slippery
ropes
from
yellowish
fangs.
+
It
was
standing
quite
still,
all
six
eyes
staring
at
them,
and
Harry
knew
that
the
only
reason
they
weren't
already
dead
was
that
their
sudden
appearance
had
taken
it
by
surprise,
but
it
was
quickly
getting
over
that,
there
was
no
mistaking
what
those
thunderous
growls
meant.
+
Harry
groped
for
the
doorknob
--
between
Filch
and
death,
he'd
take
Filch.
+
They
fell
backward
--
Harry
slammed
the
door
shut,
and
they
ran,
they
almost
flew,
back
down
the

In [19]:
len(word_pos)
len(translated_words)
processed_data_dir

'../processed_data/subject_1'

Detrending brain activations:

In [20]:
block_activations = {1:[],2:[],3:[],4:[]}
block_id = 1
for j in np.arange(0,len(activations)):
    if j > block_ends_indexes[block_id][0]:
        block_id +=1
    block_activations[block_id].append(activations[j])

print(len(block_activations[1]))
print(len(block_activations[2]))
print(len(block_activations[3]))
print(len(block_activations[4]))

clean_block_activations = {1:[],2:[],3:[],4:[]}
all_clean_activations = []
for block_id in block_activations.keys():
    print("block id:",block_id)

    clean_activations = list(block_activations[block_id])
    if fMRI_preprocessing_mode == "nothing":
        clean_activations = np.asarray(clean_activations)
    elif fMRI_preprocessing_mode == "none":
        clean_activations = np.asarray(clean_activations)
        
        indexes = np.where((np.min(clean_activations,axis=0) < np.max(clean_activations,axis=0)) == True)[0]
        indexes_p = np.where((np.min(clean_activations,axis=0) == np.max(clean_activations,axis=0)) == True)[0]

        clean_activations[:,indexes_p] = 0.0
        clean_activations[:,indexes] = \
        (clean_activations[:,indexes] - np.min(clean_activations[:,indexes],axis=0)) / \
        (np.max(clean_activations[:,indexes],axis=0) - np.min(clean_activations[:,indexes],axis=0))
    elif fMRI_preprocessing_mode == "mean":
        clean_activations = clean_activations - np.mean(clean_activations,axis=0)
    else:
        clean_activations = nilearn.signal.clean(block_activations[block_id], sessions=None, 
                             detrend=fMRI_preprocessing_flags[fMRI_preprocessing_mode]['detrend'], 
                             standardize=fMRI_preprocessing_flags[fMRI_preprocessing_mode]['standard'], 
                             confounds=None, low_pass=None, 
                             high_pass=fMRI_preprocessing_flags[fMRI_preprocessing_mode]['highpass'], 
                            t_r=2.0, ensure_finite=False)
    clean_block_activations[block_id] = clean_activations
    all_clean_activations.extend(clean_activations)

all_clean_activations = np.asarray(all_clean_activations)
print(len(all_clean_activations))

print("min:",np.min(all_clean_activations,axis=1))
print("max:",np.max(all_clean_activations,axis=1))

340
352
279
380
block id: 1
block id: 2
block id: 3
block id: 4
1351
min: [-51 -53 -49 ... -73 -73 -57]
max: [1061 1046 1024 ... 1093 1091 1101]


Saving Scan Objects:

In [21]:
block_scans = {1:[],2:[],3:[],4:[]}
block_id = 1

j = np.where(scan_times == word_times[0])[0]
for i in np.arange(0,len(actual_words)):
    if word_times[i] > scan_times[j]:
        j = np.where(scan_times == word_times[i])[0]
    if j > block_ends_indexes[block_id][0]:
            block_id +=1
    #print(block_id,word_times[i], scan_times[j][0])
    #print(word_times[i], scan_times[j])
    if word_times[i] == scan_times[j][0]:
        #print(i,j)
        
        k = i
        current_step_words = []
        current_step_pos = []
        current_step_speach_states = []
        translated_current_words = []
        counter = 0
        while k < len(actual_words) and counter < 4 :
            current_step_words.append(actual_words[k])
            current_step_pos.append(word_pos[k])
            current_step_speach_states.append(speach_states[k])
            
            translated_text_index = np.where(np.asarray(translated_block_texts_steps[block_id]) == k)
            if len(translated_text_index) > 0:
                print(block_id,translated_text_index)
                translated_current_words.append(np.asarray(translated_block_texts[block_id])[translated_text_index])
            
            k += 1
            counter +=1
            
            
        k = i+4
        next_step_words = []
        next_step_pos = []
        next_step_speach_states = []
        counter = 0
        while k >= 0 and k < len(actual_words) and counter < 4 :
            next_step_words.append(actual_words[k])
            next_step_pos.append(word_pos[k])
            next_step_speach_states.append(speach_states[k])
            k += 1
            counter +=1
         
        k = i-4
        prev_step_words = []
        prev_step_pos = []
        prev_step_speach_states = []
        counter = 0
        while k >= 0 and k < len(actual_words) and counter < 4 :
            prev_step_words.append(actual_words[k])
            prev_step_pos.append(word_pos[k])
            prev_step_speach_states.append(speach_states[k])
            k += 1
            counter +=1
            
            
        #print(translated_current_words)
        scan = Scan(all_clean_activations[j],scan_times[j][0],i,
                        prev_step_words,
                        next_step_words,
                        current_step_words,
                        current_step_pos,
                        current_step_speach_states,
                        translated_current_words) 
                   
        
        # print(i,j,block_ends_indexes[block_id][0])
        
        j += 1
        block_scans[block_id].append(scan)
        


1 (array([0]),)
1 (array([1]),)
1 (array([2]),)
1 (array([3]),)
1 (array([4]),)
1 (array([5]),)
1 (array([6]),)
1 (array([7]),)
1 (array([8]),)
1 (array([9]),)
1 (array([10]),)
1 (array([11]),)
1 (array([12]),)
1 (array([13, 14]),)
1 (array([15]),)
1 (array([16]),)
1 (array([17]),)
1 (array([18]),)
1 (array([19]),)
1 (array([20]),)
1 (array([21]),)
1 (array([22, 23]),)
1 (array([24, 25]),)
1 (array([26, 27, 28]),)
1 (array([29]),)
1 (array([30]),)
1 (array([31]),)
1 (array([32]),)
1 (array([33]),)
1 (array([34]),)
1 (array([35, 36]),)
1 (array([37]),)
1 (array([38]),)
1 (array([39]),)
1 (array([40]),)
1 (array([41]),)
1 (array([42]),)
1 (array([43]),)
1 (array([44]),)
1 (array([45]),)
1 (array([46, 47]),)
1 (array([48]),)
1 (array([49]),)
1 (array([50, 51]),)
1 (array([52]),)
1 (array([53]),)
1 (array([54]),)
1 (array([55]),)
1 (array([56]),)
1 (array([57]),)
1 (array([58]),)
1 (array([59]),)
1 (array([60]),)
1 (array([61]),)
1 (array([62]),)
1 (array([63]),)
1 (array([64]),)
1 (array(

1 (array([503, 504]),)
1 (array([505]),)
1 (array([506, 507]),)
1 (array([508]),)
1 (array([509]),)
1 (array([510]),)
1 (array([511]),)
1 (array([512]),)
1 (array([513]),)
1 (array([514]),)
1 (array([515]),)
1 (array([516]),)
1 (array([517]),)
1 (array([518]),)
1 (array([519, 520]),)
1 (array([521]),)
1 (array([522]),)
1 (array([523]),)
1 (array([524]),)
1 (array([525]),)
1 (array([526]),)
1 (array([527]),)
1 (array([528, 529]),)
1 (array([530]),)
1 (array([531]),)
1 (array([532]),)
1 (array([533]),)
1 (array([534]),)
1 (array([535]),)
1 (array([536]),)
1 (array([537]),)
1 (array([538]),)
1 (array([539]),)
1 (array([540]),)
1 (array([541, 542]),)
1 (array([543]),)
1 (array([544]),)
1 (array([545]),)
1 (array([546]),)
1 (array([547]),)
1 (array([548]),)
1 (array([549]),)
1 (array([550]),)
1 (array([551]),)
1 (array([552]),)
1 (array([553]),)
1 (array([554]),)
1 (array([555]),)
1 (array([556]),)
1 (array([557]),)
1 (array([558, 559]),)
1 (array([560]),)
1 (array([561]),)
1 (array([562]),

1 (array([1033, 1034]),)
1 (array([1035]),)
1 (array([1036, 1037]),)
1 (array([1038]),)
1 (array([1039]),)
1 (array([1040]),)
1 (array([1041]),)
1 (array([1042]),)
1 (array([1043]),)
1 (array([1044]),)
1 (array([1045]),)
1 (array([1046]),)
1 (array([1047]),)
1 (array([1048]),)
1 (array([1049]),)
1 (array([1050]),)
1 (array([1051]),)
1 (array([1052]),)
1 (array([1053]),)
1 (array([1054]),)
1 (array([1055]),)
1 (array([1056]),)
1 (array([1057]),)
1 (array([1058]),)
1 (array([1059, 1060]),)
1 (array([1061]),)
1 (array([1062]),)
1 (array([1063]),)
1 (array([1064]),)
1 (array([1065]),)
1 (array([1066]),)
1 (array([1067]),)
1 (array([1068]),)
1 (array([1069]),)
1 (array([1070]),)
1 (array([1071]),)
1 (array([1072]),)
1 (array([1073]),)
1 (array([1074]),)
1 (array([1075]),)
1 (array([1076, 1077]),)
1 (array([1078]),)
1 (array([1079]),)
1 (array([1080]),)
1 (array([1081]),)
1 (array([1082]),)
1 (array([1083]),)
1 (array([1084]),)
1 (array([1085]),)
1 (array([1086]),)
1 (array([1087, 1088]),)
1

2 (array([48]),)
2 (array([49, 50]),)
2 (array([51]),)
2 (array([52]),)
2 (array([53]),)
2 (array([54]),)
2 (array([55]),)
2 (array([56]),)
2 (array([57]),)
2 (array([58]),)
2 (array([59]),)
2 (array([60, 61]),)
2 (array([62]),)
2 (array([63]),)
2 (array([64]),)
2 (array([65, 66]),)
2 (array([67]),)
2 (array([68, 69]),)
2 (array([70]),)
2 (array([71, 72]),)
2 (array([73]),)
2 (array([74]),)
2 (array([75]),)
2 (array([76]),)
2 (array([77]),)
2 (array([78]),)
2 (array([79]),)
2 (array([80]),)
2 (array([81]),)
2 (array([82]),)
2 (array([83, 84]),)
2 (array([85, 86]),)
2 (array([87]),)
2 (array([88]),)
2 (array([89, 90]),)
2 (array([91, 92]),)
2 (array([93]),)
2 (array([94]),)
2 (array([95]),)
2 (array([96]),)
2 (array([97, 98]),)
2 (array([99]),)
2 (array([100, 101, 102]),)
2 (array([103]),)
2 (array([104]),)
2 (array([105, 106]),)
2 (array([107, 108]),)
2 (array([109]),)
2 (array([110]),)
2 (array([111]),)
2 (array([112]),)
2 (array([113]),)
2 (array([114]),)
2 (array([115]),)
2 (array([

2 (array([713]),)
2 (array([714]),)
2 (array([715, 716]),)
2 (array([717]),)
2 (array([718]),)
2 (array([719]),)
2 (array([720]),)
2 (array([721, 722]),)
2 (array([723]),)
2 (array([724]),)
2 (array([725]),)
2 (array([726]),)
2 (array([727, 728]),)
2 (array([729]),)
2 (array([730]),)
2 (array([731]),)
2 (array([732]),)
2 (array([733]),)
2 (array([734]),)
2 (array([735]),)
2 (array([736]),)
2 (array([737]),)
2 (array([738, 739]),)
2 (array([740]),)
2 (array([741]),)
2 (array([742]),)
2 (array([743]),)
2 (array([744]),)
2 (array([745]),)
2 (array([746]),)
2 (array([747]),)
2 (array([748]),)
2 (array([749]),)
2 (array([750]),)
2 (array([751]),)
2 (array([752]),)
2 (array([753, 754]),)
2 (array([755]),)
2 (array([756]),)
2 (array([757]),)
2 (array([758]),)
2 (array([759]),)
2 (array([760]),)
2 (array([761, 762]),)
2 (array([763]),)
2 (array([764]),)
2 (array([765]),)
2 (array([766, 767]),)
2 (array([768]),)
2 (array([769]),)
2 (array([770]),)
2 (array([771]),)
2 (array([772]),)
2 (array([7

...

3 (array([382]),)
3 (array([383]),)
3 (array([384]),)
3 (array([385]),)
3 (array([386]),)
3 (array([387]),)
3 (array([388]),)
3 (array([389, 390]),)
3 (array([391]),)
3 (array([392, 393]),)
3 (array([394]),)
3 (array([395]),)
3 (array([396]),)
3 (array([397]),)
3 (array([398]),)
3 (array([399]),)
3 (array([400]),)
3 (array([401, 402]),)
3 (array([403, 404]),)
3 (array([405]),)
3 (array([406]),)
3 (array([407]),)
3 (array([408]),)
3 (array([409]),)
3 (array([410]),)
3 (array([411]),)
3 (array([412]),)
3 (array([413]),)
3 (array([414]),)
3 (array([415]),)
3 (array([416, 417]),)
3 (array([418]),)
3 (array([419, 420]),)
3 (array([421]),)
3 (array([422, 423]),)
3 (array([424]),)
3 (array([425]),)
3 (array([426]),)
3 (array([427]),)
3 (array([428, 429]),)
3 (array([430]),)
3 (array([431]),)
3 (array([432]),)
3 (array([433]),)
3 (array([434]),)
3 (array([435]),)
3 (array([436, 437]),)
3 (array([438]),)
3 (array([439]),)
3 (array([440]),)
3 (array([441]),)
3 (array([442]),)
3 (array([443]),)
3

3 (array([1025]),)
3 (array([1026, 1027]),)
3 (array([1028]),)
3 (array([1029, 1030]),)
3 (array([1031, 1032]),)
3 (array([1033]),)
3 (array([1034, 1035, 1036]),)
3 (array([1037]),)
3 (array([1038, 1039]),)
3 (array([1040]),)
3 (array([1041]),)
3 (array([1042]),)
3 (array([1043]),)
3 (array([1044]),)
3 (array([1045]),)
3 (array([1046]),)
3 (array([1047]),)
3 (array([1048]),)
3 (array([1049]),)
3 (array([1050]),)
3 (array([1051]),)
3 (array([1052]),)
3 (array([1053, 1054]),)
3 (array([1055]),)
3 (array([1056]),)
3 (array([1057]),)
3 (array([1058]),)
3 (array([1059]),)
3 (array([1060]),)
3 (array([1061]),)
3 (array([1062]),)
3 (array([1063]),)
3 (array([1064]),)
3 (array([1065]),)
3 (array([1066, 1067]),)
3 (array([1068]),)
3 (array([1069]),)
3 (array([1070]),)
3 (array([1071]),)
3 (array([1072]),)
3 (array([1073]),)
3 (array([1074, 1075]),)
3 (array([1076]),)
3 (array([1077]),)
3 (array([1078]),)
3 (array([1079]),)
3 (array([1080]),)
3 (array([1081, 1082]),)
3 (array([1083]),)
3 (array(

4 (array([282]),)
4 (array([283]),)
4 (array([284]),)
4 (array([285]),)
4 (array([286]),)
4 (array([287]),)
4 (array([288]),)
4 (array([289]),)
4 (array([290, 291]),)
4 (array([292]),)
4 (array([293]),)
4 (array([294]),)
4 (array([295]),)
4 (array([296]),)
4 (array([297, 298]),)
4 (array([299]),)
4 (array([300]),)
4 (array([301]),)
4 (array([302, 303]),)
4 (array([304]),)
4 (array([305]),)
4 (array([306]),)
4 (array([307]),)
4 (array([308]),)
4 (array([309]),)
4 (array([310]),)
4 (array([311]),)
4 (array([312]),)
4 (array([313]),)
4 (array([314]),)
4 (array([315]),)
4 (array([316]),)
4 (array([317]),)
4 (array([318]),)
4 (array([319, 320]),)
4 (array([321]),)
4 (array([322, 323]),)
4 (array([324]),)
4 (array([325]),)
4 (array([326, 327]),)
4 (array([328]),)
4 (array([329]),)
4 (array([330]),)
4 (array([331, 332]),)
4 (array([333, 334]),)
4 (array([335, 336]),)
4 (array([337]),)
4 (array([338, 339]),)
4 (array([340, 341]),)
4 (array([342]),)
4 (array([343]),)
4 (array([344]),)
4 (array(

4 (array([978, 979]),)
4 (array([980, 981, 982]),)
4 (array([983]),)
4 (array([984, 985]),)
4 (array([986, 987]),)
4 (array([988, 989, 990, 991]),)
4 (array([992]),)
4 (array([993, 994]),)
4 (array([995]),)
4 (array([996]),)
4 (array([997, 998]),)
4 (array([ 999, 1000]),)
4 (array([1001]),)
4 (array([1002, 1003]),)
4 (array([1004, 1005]),)
4 (array([1006, 1007]),)
4 (array([1008, 1009, 1010]),)
4 (array([1011, 1012]),)
4 (array([1013]),)
4 (array([1014]),)
4 (array([1015]),)
4 (array([1016]),)
4 (array([1017]),)
4 (array([1018]),)
4 (array([1019, 1020]),)
4 (array([1021]),)
4 (array([1022]),)
4 (array([1023]),)
4 (array([1024]),)
4 (array([1025]),)
4 (array([1026]),)
4 (array([1027, 1028]),)
4 (array([1029, 1030]),)
4 (array([1031]),)
4 (array([1032]),)
4 (array([1033, 1034, 1035]),)
4 (array([1036]),)
4 (array([1037, 1038, 1039]),)
4 (array([1040]),)
4 (array([1041, 1042]),)
4 (array([1043]),)
4 (array([1044]),)
4 (array([1045]),)
4 (array([1046]),)
4 (array([1047]),)
4 (array([1048])

4 (array([1569]),)
4 (array([1570]),)
4 (array([1571]),)
4 (array([1572, 1573]),)
4 (array([1574]),)
4 (array([1575, 1576]),)
4 (array([1577, 1578]),)
4 (array([1579]),)
4 (array([1580, 1581]),)
4 (array([1582, 1583]),)
4 (array([1584]),)
4 (array([1585]),)
4 (array([1586]),)
4 (array([1587]),)
4 (array([1588, 1589]),)
4 (array([1590]),)
4 (array([1591]),)
4 (array([1592]),)
4 (array([1593]),)
4 (array([1594]),)
4 (array([1595]),)
4 (array([1596, 1597]),)
4 (array([1598]),)
4 (array([1599, 1600, 1601]),)
4 (array([1602, 1603]),)
4 (array([1604]),)
4 (array([1605, 1606]),)
4 (array([1607]),)
4 (array([1608]),)
4 (array([1609]),)
4 (array([1610]),)
4 (array([1611]),)
4 (array([1612, 1613]),)
4 (array([1614]),)
4 (array([1615]),)
4 (array([1616]),)
4 (array([1617, 1618]),)
4 (array([1619]),)
4 (array([1620]),)
4 (array([1621]),)
4 (array([1622, 1623]),)
4 (array([1624]),)
4 (array([1625]),)
4 (array([1626, 1627]),)
4 (array([1628]),)
4 (array([1629, 1630]),)
4 (array([1631]),)
4 (array([1

In [22]:
translated_block_texts[2][0]

'"'

In [23]:
translated_block_texts_steps[1]
print(len(block_text[4].split()))
print(len(translated_block_texts[4]))

for i in np.arange(len(block_text[1].split())):
    if block_text[1].split()[i] != translated_block_texts[1][i]:
        print(block_text[1].split()[i],translated_block_texts[1][i])


1766
1766


In [24]:
block_scans[1][0].current_translated_words

[array(['Harry'], dtype='<U13'),
 array(['had'], dtype='<U13'),
 array(['never'], dtype='<U13'),
 array(['believed'], dtype='<U13')]

In [25]:
np.save(processed_data_dir+"/"+fMRI_preprocessing_mode+"subject_"+subject_id+"_scan_objects",block_scans)

In [26]:
block_id = 1
print(translated_block_texts_steps[block_id])
np.where(np.asarray(translated_block_texts_steps[block_id]) == 1)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 14, 15, 16, 17, 18, 19, 20, 21, 21, 22, 22, 23, 23, 23, 24, 25, 26, 27, 28, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 40, 41, 42, 43, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 78, 79, 80, 80, 80, 81, 82, 83, 83, 84, 84, 85, 86, 87, 88, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 115, 116, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 125, 126, 127, 128, 128, 129, 129, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 142, 143, 144, 145, 146, 147, 148, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 187, 188, 189, 190, 191, 192, 192, 193, 193, 194, 195,

(array([1]),)

After running the language model on the stimuli, reading and saving the hidden states as embeddings:

In [27]:
for layer_id in np.arange(2):
    embeddings = {1:{},2:{},3:{},4:{}}
    for block_id in np.arange(1,5):
        
        print("phrases: ",len(translated_block_texts_steps[block_id]),translated_block_texts_steps[block_id][-1])
        print("scans: ",len(block_scans[block_id]), block_scans[block_id][-1].step)
        count = 0
        for scan in block_scans[block_id]:            
            step = scan.step
            real_step = step
            while real_step not in translated_block_texts_steps[block_id] and (real_step - step) < 3:
                real_step += 1
                print(real_step)
                
            if real_step in translated_block_texts_steps[block_id]:
                count += 1              
                embeddings[block_id][step] = []
                i = 0
                while i < 4 and (real_step+i) in translated_block_texts_steps[block_id]:
                    translated_steps = np.where(np.asarray(translated_block_texts_steps[block_id]) == real_step+i)[0]
                    print("translated steps %s, step %d, word %s" %(translated_steps,step,scan.all_words))
                    one_step_embeddings = []
                    for translated_step in translated_steps:
                        lstm_emb= np.load("../../lm_1b/new_output/subject_"+subject_id+"_block_"+str(block_id)+"_lstm_"+str(layer_id)+"_embeddings/lstm_emb_step_"+str(translated_step+1)+".npy")
                        print(lstm_emb)
                        one_step_embeddings.append(lstm_emb[0])
                    
                    embeddings[block_id][step].append(np.mean(one_step_embeddings,axis=0))
                    print(np.asarray(embeddings[block_id][step]).shape)
                    i += 1
                
        print(count)
        print(len(embeddings[block_id]))
        #print(embeddings[block_id].keys())
    np.save("../embeddings/subject_"+subject_id+"_lstm_"+str(layer_id)+"_emb_objects",embeddings)
    print(len(embeddings[1][0]))

phrases:  1519 1302
scans:  326 1300
translated steps [0], step 0, word ['Harry', 'had', 'never', 'believed']
[[-0.0872765   0.04032643 -0.45139223 ... -0.10783331  0.02813069
  -0.12154386]]
(1, 1024)
translated steps [1], step 0, word ['Harry', 'had', 'never', 'believed']
[[-0.03125925 -0.2477504  -0.03791803 ... -0.40889603  0.07107337
  -0.13472961]]
(2, 1024)
translated steps [2], step 0, word ['Harry', 'had', 'never', 'believed']
[[-0.36215922 -0.9615818  -0.21803743 ...  0.10343758  0.4459945
   0.25937   ]]
(3, 1024)
translated steps [3], step 0, word ['Harry', 'had', 'never', 'believed']
[[-0.8043436  -0.600949    0.3271572  ... -1.068557    0.8109199
  -0.03367356]]
(4, 1024)
translated steps [4], step 4, word ['he', 'would', 'meet', 'a']
[[ 0.4679604  -0.6901602  -0.00552494 ... -0.6177721   0.45266
  -0.21184509]]
(1, 1024)
translated steps [5], step 4, word ['he', 'would', 'meet', 'a']
[[-0.274165   -0.73976    -0.55999136 ... -0.13215657  0.23977607
   0.32054666]]
(2, 10

[[ 0.49698994  0.5815984  -0.14454858 ... -0.18770427  0.1360281
   0.24327514]]
(3, 1024)
translated steps [59], step 48, word ['spotted', 'a', 'notice', 'pinned']
[[-0.61105853  0.02959094 -0.98691994 ... -0.55249226  0.53511184
  -0.5964625 ]]
(4, 1024)
translated steps [60], step 52, word ['up', 'in', 'the', 'Gryffindor']
[[-0.04075192  0.33440706 -0.14474346 ...  0.09686066 -0.26849535
   0.10120533]]
(1, 1024)
translated steps [61], step 52, word ['up', 'in', 'the', 'Gryffindor']
[[-0.11417457 -0.17248519  0.13726646 ...  0.31120923  0.02301223
  -0.01265299]]
(2, 1024)
translated steps [62], step 52, word ['up', 'in', 'the', 'Gryffindor']
[[ 0.03555911 -0.08425073 -0.15848003 ...  0.00147223  0.23519
   0.06147645]]
(3, 1024)
translated steps [63], step 52, word ['up', 'in', 'the', 'Gryffindor']
[[-0.11500067  0.21568379 -0.5509891  ... -0.01101984 -0.05673677
  -0.7473974 ]]
(4, 1024)
translated steps [64], step 56, word ['common', 'room', 'that', 'made']
[[-0.7287308  -0.53400

[[ 0.01690101 -0.15712796  0.08490984 ... -0.04878801  0.43630213
   0.1482208 ]]
(1, 1024)
translated steps [159], step 136, word ['about', 'how', 'good', 'he']
[[-0.1794121  -0.47179148  0.32178104 ... -0.81662416  0.07629813
   0.10979934]]
(2, 1024)
translated steps [160], step 136, word ['about', 'how', 'good', 'he']
[[-0.71505886  0.41401798 -0.9224829  ...  0.08994998 -0.22687578
   0.49032572]]
(3, 1024)
translated steps [161], step 136, word ['about', 'how', 'good', 'he']
[[ 0.12124285 -0.15252963  0.2934124  ... -1.0403221  -0.11965227
  -0.18591744]]
(4, 1024)
translated steps [162], step 140, word ['is', 'at', 'Quidditch,', 'but']
[[ 0.29174978  0.5652086  -0.07924465 ... -0.05292292  0.4596689
  -0.3304506 ]]
(1, 1024)
translated steps [163], step 140, word ['is', 'at', 'Quidditch,', 'but']
[[-0.01181401 -0.09783555 -0.6009371  ... -0.33215997  0.2586369
  -0.36733618]]
(2, 1024)
translated steps [164 165], step 140, word ['is', 'at', 'Quidditch,', 'but']
[[-0.92997396  0.

[[ 0.9365318  -1.2411999  -0.3506509  ... -0.31521013 -0.23879379
   0.44123816]]
(2, 1024)
translated steps [223], step 192, word ['one,', 'though:', 'the', 'way']
[[ 0.33806342 -0.56334144 -0.0888944  ... -0.02535173  0.08211532
  -0.07712486]]
(3, 1024)
translated steps [224], step 192, word ['one,', 'though:', 'the', 'way']
[[-0.16923435 -0.88393897  0.0101722  ...  0.06876092  0.9227538
  -0.39812055]]
(4, 1024)
translated steps [225], step 196, word ['Seamus', 'Finnigan', 'told', 'it,']
[[-1.1205068  -0.13304913  1.1621914  ... -0.9610924   0.14136697
   0.09204999]]
(1, 1024)
translated steps [226], step 196, word ['Seamus', 'Finnigan', 'told', 'it,']
[[-0.59287035  0.0769317  -0.02568275 ... -0.30319917 -0.04774456
   0.11483522]]
(2, 1024)
translated steps [227], step 196, word ['Seamus', 'Finnigan', 'told', 'it,']
[[ 0.3056467   0.21817932  0.17182325 ... -0.02938718  0.3003202
   0.49498177]]
(3, 1024)
translated steps [228 229], step 196, word ['Seamus', 'Finnigan', 'told',

[[ 0.7366208   0.1934088  -0.2797819  ...  0.93172956 -0.25120574
  -0.35650408]]
[[ 0.16142404  0.54064584 -0.14936824 ...  0.07927683  0.51211226
  -0.08926363]]
(3, 1024)
translated steps [290], step 252, word ['shared', 'their', 'dormitory,', 'about']
[[ 0.09554685 -0.0459066   0.24963786 ... -0.1492521   0.7531044
   0.16040866]]
(4, 1024)
translated steps [291 292], step 256, word ['soccer.', 'Ron', "couldn't", 'see']
[[-0.48838884  1.1249446   1.1150252  ...  0.48546094 -0.26823354
   0.21431303]]
[[0.5162891  0.08449158 1.3972718  ... 0.41017005 0.20404164 0.59300995]]
(1, 1024)
translated steps [293], step 256, word ['soccer.', 'Ron', "couldn't", 'see']
[[ 0.6695647   0.22416058  0.5448469  ... -0.058773   -0.4920023
  -0.00912523]]
(2, 1024)
translated steps [294], step 256, word ['soccer.', 'Ron', "couldn't", 'see']
[[ 0.00181902 -0.6665106  -0.20485547 ...  0.6823055  -0.17734519
  -0.47805193]]
(3, 1024)
translated steps [295], step 256, word ['soccer.', 'Ron', "couldn't",

...

[[ 1.0187854  -0.19673362  0.4933228  ...  0.71029633  0.03305532
  -0.02932089]]
(2, 1024)
translated steps [1425], step 4897, word ['looking', 'at', 'their', 'bathrobes']
[[ 1.0453763  -0.6282016   0.66460377 ...  1.0625482  -0.5272524
   0.41296014]]
(3, 1024)
translated steps [1426], step 4897, word ['looking', 'at', 'their', 'bathrobes']
[[ 0.44619423  0.26057333 -0.22977357 ...  1.0414084  -0.27210075
  -0.37634632]]
(4, 1024)
translated steps [1427], step 4901, word ['hanging', 'off', 'their', 'shoulders']
[[-0.197312    0.6442141  -0.09384928 ...  0.6654151  -0.37235114
  -0.14640772]]
(1, 1024)
translated steps [1428], step 4901, word ['hanging', 'off', 'their', 'shoulders']
[[ 0.8247079   0.06900364  0.7073356  ...  1.1477509  -0.2593845
  -0.666853  ]]
(2, 1024)
translated steps [1429], step 4901, word ['hanging', 'off', 'their', 'shoulders']
[[ 0.47445112 -0.94261134  0.99295145 ...  0.63596237 -0.05030236
  -0.02788519]]
(3, 1024)
translated steps [1430], step 4901, word [

[[-0.29904482  0.3121993   0.74463177 ... -0.26894554  0.21329965
  -0.09523972]]
(3, 1024)
translated steps [1506], step 4961, word ['think', "they're", 'doing,', 'keeping']
[[ 0.02652352  0.15151286  0.79773283 ... -0.65129757  0.41704595
  -0.31857488]]
(4, 1024)
translated steps [1507], step 4965, word ['a', 'thing', 'like', 'that']
[[ 0.47555453 -0.04754731  0.7013588  ... -0.2035354   0.06824035
  -0.12570325]]
(1, 1024)
translated steps [1508], step 4965, word ['a', 'thing', 'like', 'that']
[[ 0.34150663 -0.07160688  0.5356426  ... -0.57952607  0.08122429
  -0.07894655]]
(2, 1024)
translated steps [1509], step 4965, word ['a', 'thing', 'like', 'that']
[[ 0.05186643 -0.50376415  0.5809301  ... -0.6450221   0.22846386
  -0.34198135]]
(3, 1024)
translated steps [1510], step 4965, word ['a', 'thing', 'like', 'that']
[[ 0.46444955  0.13644308  0.7737119  ... -0.5778358   0.44400424
  -0.13415143]]
(4, 1024)
translated steps [1511], step 4969, word ['locked', 'up', 'in', 'a']
[[ 0.097

(2, 1024)
translated steps [1580 1581], step 5021, word ['floor?"', 'Harry', 'suggested.', '"I']
[[ 0.26252207 -0.08674224 -0.05253099 ... -0.01897931 -0.21336144
  -0.17810102]]
[[ 0.37796104 -0.72228175  0.26661128 ... -0.3242051  -0.24783897
  -0.26112396]]
(3, 1024)
translated steps [1582 1583], step 5021, word ['floor?"', 'Harry', 'suggested.', '"I']
[[ 0.01774993 -0.9172905   0.37684926 ... -0.3307563  -0.38394293
  -0.23446363]]
[[-0.31357604  0.27785382  0.0318358  ... -0.09807569 -0.00109848
   0.3711573 ]]
(4, 1024)
translated steps [1584], step 5025, word ["wasn't", 'looking', 'at', 'its']
[[ 0.13878565  0.18897869 -0.03142865 ... -0.115708   -0.0112759
   0.22156408]]
(1, 1024)
translated steps [1585], step 5025, word ["wasn't", 'looking', 'at', 'its']
[[ 0.16885448 -0.33156863 -0.40039292 ... -0.14687362 -0.21893337
   0.08352103]]
(2, 1024)
translated steps [1586], step 5025, word ["wasn't", 'looking', 'at', 'its']
[[ 0.31176457  0.08313012 -0.09451856 ...  0.44608918 -0.

[[ 1.0421519  -0.4101617   0.03016506 ... -0.05809458 -0.63206553
  -0.62935853]]
(3, 1024)
translated steps [1649 1650], step 5073, word ['or', 'worse,', 'expelled.', 'Now,']
[[ 0.45237428 -0.18088488  0.17941135 ...  0.63727385 -0.83754903
  -0.47019982]]
[[ 0.6561834  -0.45698106  0.95540357 ... -0.01095105 -0.5695185
  -0.24244167]]
(4, 1024)
translated steps [1651], step 5077, word ['if', 'you', "don't", 'mind,']
[[ 0.5825804  -0.16754982  0.46779838 ... -0.53649664 -0.190156
  -0.09112424]]
(1, 1024)
translated steps [1652], step 5077, word ['if', 'you', "don't", 'mind,']
[[-0.00663869 -0.2627311   0.03251737 ...  0.05742911 -0.07368204
   0.157697  ]]
(2, 1024)
translated steps [1653], step 5077, word ['if', 'you', "don't", 'mind,']
[[ 0.80893594 -0.3061466  -0.07340678 ...  0.28917456 -0.32155612
  -0.04060712]]
(3, 1024)
translated steps [1654 1655], step 5077, word ['if', 'you', "don't", 'mind,']
[[ 1.3451774  -0.39228237  0.11442471 ...  1.0492637  -0.5498566
  -0.14962384]]

[[ 0.43432286 -0.42582634  0.43133345 ...  0.23845568  0.7143096
  -0.2719675 ]]
[[ 0.12963618 -0.21687277  0.48368886 ... -0.03797467  0.09839713
  -0.3077204 ]]
(3, 1024)
translated steps [1724], step 5133, word ['had', 'Hagrid', 'said?', 'Gringotts']
[[ 0.04964872 -0.26618174  0.0580594  ... -0.0096407   0.40822506
  -0.20943679]]
(4, 1024)
translated steps [1725], step 5137, word ['was', 'the', 'safest', 'place']
[[ 0.7617887  -0.40006438  0.02497926 ...  0.19472513  0.11966982
  -0.28878963]]
(1, 1024)
translated steps [1726], step 5137, word ['was', 'the', 'safest', 'place']
[[ 0.53382695 -0.4127744   0.43183288 ...  0.78182936  0.06118667
  -0.19284645]]
(2, 1024)
translated steps [1727], step 5137, word ['was', 'the', 'safest', 'place']
[[ 0.698332   -0.45061564  0.26122886 ...  0.20548739  0.4105389
   0.3065745 ]]
(3, 1024)
translated steps [1728], step 5137, word ['was', 'the', 'safest', 'place']
[[ 1.037214   -0.34521765  0.2647596  ... -0.14932323  0.30273592
  -0.347671  

In [28]:
vocab_set = list(set(vocab))

In [29]:
len(vocab_set)

1837

In [30]:
#read words:
with open("subject_"+subject_id+"_harry_potter_vocab.txt","r") as fin:
    print("Reading words...")
    words = fin.readlines()
    for i in np.arange(len(words)):
        words[i] = words[i].replace('\n',"")

Reading words...


In [31]:
word_embeddings = np.load("../embeddings/subject_1/embeddings_char_cnn.npy")#("embeddings_char_cnn.npy")
len(word_embeddings)

5176

In [32]:
word_embedding_dic = {}
for i in np.arange(len(words)):
    word_embedding_dic[words[i]] = word_embeddings[i]
    
np.save("../embeddings/subject_"+subject_id+"_char_cnn_embedding_dic",word_embedding_dic)

In [33]:
softmax_embeddings = np.load("../embeddings/subject_1/embeddings_softmax.npy")

In [34]:
softmax_embeddings.shape

(800000, 1024)

In [35]:
all_vocab = []
softmax_vocab = []
with open("../../lm_1b/data/vocab-2016-09-10.txt","r") as vf:
    lines = vf.readlines()
    for line in lines:
        softmax_vocab.append(line.replace('\n',''))

In [36]:
softmax_word_embedding_dic = {}
for i in np.arange(len(words)):
    if words[i] in softmax_vocab:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index(words[i])]
    elif words[i].lower() in softmax_vocab:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index(words[i].lower())]
    else:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index('<UNK>')]
        
np.save("../embeddings/subject_"+subject_id+"_softmax_embedding_dic",word_embedding_dic)

In [37]:
coords[:,0]

array([ 3,  3,  3, ..., 51, 51, 51], dtype=uint8)

In [38]:
brain3d = np.zeros((np.max(coords[:,0])+1,np.max(coords[:,1])+1,np.max(coords[:,2])+1))

In [39]:
block_scans[1][2]

In [40]:
block_id = 1
for k in np.arange(len(block_scans[block_id])):
    brain3d = np.zeros((np.max(coords[:,0])+1,np.max(coords[:,1])+1,np.max(coords[:,2])+1))    
    for i in np.arange(len(coords)):
        brain3d[coords[i][0],coords[i][1],coords[i][2]] = block_scans[block_id][k].activations[0][i]
    block_scans[block_id][k].brain3d = brain3d

In [41]:
x,y,z = block_scans[1][0].brain3d.nonzero()

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, -z, zdir='z', c= 'red')
plt.savefig("demo.png")


In [42]:
np.save(processed_data_dir+"/subject_"+subject_id+"_extended_scan_object",block_scans)

In [43]:
loaded_extended_scans = np.load(processed_data_dir+"/subject_"+subject_id+"_extended_scan_object.npy")

print(loaded_extended_scans.item().keys())
block_id = 1
print(len(loaded_extended_scans.item()[block_id]))
i = 0
print(loaded_extended_scans.item()[block_id][i].all_words)

dict_keys([1, 2, 3, 4])
326
['Harry', 'had', 'never', 'believed']


In [44]:
scan_objects = np.load("../processed_data/subject_2/subject_"+str(subject_id)+"_scan_objects.npy")
glove_embeddings = np.load("../embeddings/glove_word_embedding_dic.npy")
glove_embeddings.item()[""] = np.zeros_like(glove_embeddings.item()[list(glove_embeddings.item().keys())[0]])
glove_embeddings.item()["+"] = np.zeros_like(glove_embeddings.item()[list(glove_embeddings.item().keys())[0]])

FileNotFoundError: [Errno 2] No such file or directory: '../processed_data/subject_2/subject_1_scan_objects.npy'

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

block_ids = [1,2,3,4]

filtered_glove_embedding_dic = {}
filtered_glove_embedding_step_dic = {1:{},2:{},3:{},4:{}}
for block_id in block_ids:
    for scan_obj in scan_objects.item().get(block_id):
        words = []
        current_glove_words = []
        current_glove_embeddings = []
        for wordgroup in scan_obj.current_translated_words:
            words.extend(wordgroup)
            
                    
        if set(words).issubset(glove_embeddings.item().keys()):
            current_glove_words =  words
            for dic_word in current_glove_words:
                filtered_glove_embedding_dic[dic_word] = glove_embeddings.item()[dic_word]
        else:
            for w in words:
                if w in glove_embeddings.item().keys():
                    current_glove_words.append(w)
                    filtered_glove_embedding_dic[w] = glove_embeddings.item()[w]
                elif w.lower() in glove_embeddings.item().keys():
                    current_glove_words.append(w.lower())
                    filtered_glove_embedding_dic[w] = glove_embeddings.item()[w.lower()]
                else:
                    tokenized_w = nltk.word_tokenize(w.lower())
                    if set(tokenized_w).issubset(glove_embeddings.item().keys()):
                        current_glove_words.extend(tokenized_w)
                        for tok_w in tokenized_w:
                             filtered_glove_embedding_dic[w] = glove_embeddings.item()[tok_w]
                    else:
                        for t_w in tokenized_w:
                            if t_w in glove_embeddings.item().keys():
                                current_glove_words.append(t_w)
                                filtered_glove_embedding_dic[w] = glove_embeddings.item()[t_w]
                            else:
                                l_t_w = lemmatizer.lemmatize(t_w)
                                if l_t_w in glove_embeddings.item().keys():
                                    current_glove_words.append(l_t_w)
                                    filtered_glove_embedding_dic[w] = glove_embeddings.item()[l_t_w]
                                else:
                                    s_l_t_w = ps.stem(l_t_w)
                                    if s_l_t_w in glove_embeddings.item().keys():
                                        current_glove_words.append(s_l_t_w)
                                        filtered_glove_embedding_dic[w] = glove_embeddings.item()[s_l_t_w]
                                    else:
                                        print(w)
                                        filtered_glove_embedding_dic[w] = glove_embeddings.item()['unk']

        for w in current_glove_words:
            current_glove_embeddings.append(glove_embeddings.item()[w])
            
        filtered_glove_embedding_step_dic[block_id][scan_obj.step] = (current_glove_words,current_glove_embeddings)
            

In [ ]:

np.save("../embeddings/filtered_glove_embedding_step_dic",filtered_glove_embedding_step_dic)
np.save("../embeddings/filtered_glove_embedding_dic",filtered_glove_embedding_dic)

In [ ]:
glove_embeddings.item()['unk']